## Mixed Model NBA Shooting Predictor

#### Table of contents
    1. Motivation
    2. Exploratory Data Analysis

#### Motivation

The goal of this project to is to build a model that predicts the percentage of making a field goal. This project will explore three seperate types of models to compare which best fits this scenario. 

The first model I will explore will be a fixed effect model such as gradient boosting. Each factor will have the same coefficients in the model no matter the player or group.

The second model I will explore will be a Random effect model such as variance component analysis. Each 'observation' will have a different level of varaiblity. Our model will have different coefficients for each player or group.

Lastly we will look at mixed models that contain both random effects and fixed effects. 


In [2]:
#import libraries
import matplotlib.pyplot as plt
import numpy as np
import time
import pandas as pd
from py_ball import player, playbyplay, image, league_dash, team, boxscore
from sklearn.model_selection import train_test_split

In [2]:
#to build connection to py_ball library
HEADERS = {'Connection': 'keep-alive',
           'Host': 'stats.nba.com',
           'Origin': 'http://stats.nba.com',
           'Upgrade-Insecure-Requests': '1',
           'Referer': 'stats.nba.com',
           'x-nba-stats-origin': 'stats',
           'x-nba-stats-token': 'true',
           'Accept-Language': 'en-US,en;q=0.9',
           "X-NewRelic-ID": "VQECWF5UChAHUlNTBwgBVw==",
           'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6)' +\
                         ' AppleWebKit/537.36 (KHTML, like Gecko)' + \
                         ' Chrome/81.0.4044.129 Safari/537.36'}

#### Exploratory Data Analysis

I first explore what data is available through the py_ball api. The first goal is to use the data available to determine which features could contribute to a players field goal percentage during a game. 

The players location on the floor, the angle they shoot from, the type of shot being taken, and the opposing teams defensive rating will make a good starting base for our models features.

In [ ]:
#Building a dataframe of the shots taken during the regular season of the 22-23 season, saved as shotchart.csv
league_id = '00' #NBA
player_id = '0' #All players

season = '2022-23'
x=season.split('0',1)[1]
season_split = x.split('-',1)[0]
all_df = pd.DataFrame({})

for x in range(1,1231):
    print('Game #' + str(x))
    t0 = time.time()
    game_id = '002'+ season_split +'0'+ str(x).zfill(4)
    shots = player.Player(headers=HEADERS,
                      endpoint='shotchartdetail',
                      league_id=league_id,
                      player_id=player_id,
                      game_id=game_id,
                      season=season)
    shot_df = pd.DataFrame(shots.data['Shot_Chart_Detail'])
    all_df = pd.concat([all_df, shot_df], axis=0).reset_index(drop=True)
    delay = time.time() - t0
    print('Waiting ' + str(10*delay) + 's')
    time.sleep(delay)
    
all_df.to_csv('shotchart.csv', index=False)

In [5]:
pd.set_option('display.max_columns', None) # displays all columns of the shotchart
shot_chart = pd.read_csv(r'/home/neil/Desktop/shotchart.csv')
box_score = pd.read_csv(r'/home/neil/Desktop/game_box_scores_2223_Team.csv')

In [7]:
shot_chart

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,22200001,7,203954,Joel Embiid,1610612755,Philadelphia 76ers,1,11,38,Missed Shot,Turnaround Fadeaway shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,12,-118,50,1,0,20221018,BOS,PHI
1,Shot Chart Detail,22200001,11,203935,Marcus Smart,1610612738,Boston Celtics,1,11,15,Made Shot,Driving Floating Bank Jump Shot,2PT Field Goal,Mid-Range,Right Side(R),8-16 ft.,13,120,55,1,1,20221018,BOS,PHI
2,Shot Chart Detail,22200001,12,202699,Tobias Harris,1610612755,Philadelphia 76ers,1,11,5,Missed Shot,Driving Floating Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,14,50,135,1,0,20221018,BOS,PHI
3,Shot Chart Detail,22200001,14,202699,Tobias Harris,1610612755,Philadelphia 76ers,1,11,3,Made Shot,Tip Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,0,0,0,1,1,20221018,BOS,PHI
4,Shot Chart Detail,22200001,15,1628369,Jayson Tatum,1610612738,Boston Celtics,1,10,46,Made Shot,Jump Shot,3PT Field Goal,Left Corner 3,Left Side(L),24+ ft.,23,-232,49,1,1,20221018,BOS,PHI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217215,Shot Chart Detail,22201230,634,1630541,Moses Moody,1610612744,Golden State Warriors,4,1,11,Made Shot,Cutting Dunk Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,11,3,1,1,20230409,POR,GSW
217216,Shot Chart Detail,22201230,636,1631219,John Butler Jr.,1610612757,Portland Trail Blazers,4,1,0,Made Shot,Jump Shot,3PT Field Goal,Right Corner 3,Right Side(R),24+ ft.,23,224,84,1,1,20230409,POR,GSW
217217,Shot Chart Detail,22201230,638,1630228,Jonathan Kuminga,1610612744,Golden State Warriors,4,0,44,Missed Shot,Pullup Jump shot,2PT Field Goal,Mid-Range,Right Side(R),16-24 ft.,23,199,126,1,0,20230409,POR,GSW
217218,Shot Chart Detail,22201230,640,1631116,Patrick Baldwin,1610612744,Golden State Warriors,4,0,36,Missed Shot,Driving Floating Bank Jump Shot,2PT Field Goal,In The Paint (Non-RA),Left Side(L),8-16 ft.,8,-71,52,1,0,20230409,POR,GSW


In [5]:
shot_chart['ACTION_TYPE'].value_counts()

ACTION_TYPE
Jump Shot                             63981
Pullup Jump shot                      27214
Driving Layup Shot                    19169
Driving Floating Jump Shot            11246
Step Back Jump shot                   10457
Running Layup Shot                     6766
Layup Shot                             5824
Cutting Layup Shot                     5518
Driving Finger Roll Layup Shot         5516
Tip Layup Shot                         4685
Running Jump Shot                      4398
Floating Jump shot                     4331
Fadeaway Jump Shot                     3923
Putback Layup Shot                     3525
Driving Floating Bank Jump Shot        3356
Cutting Dunk Shot                      3340
Turnaround Fadeaway shot               3214
Turnaround Jump Shot                   2868
Running Pull-Up Jump Shot              2601
Running Dunk Shot                      2569
Turnaround Hook Shot                   2546
Driving Reverse Layup Shot             2218
Driving Dunk Shot   

In [6]:
#from basketball_relativity
def feature_engineering(shotchart_df):
    """ feature_engineering calculates engineered
    features from the shotchart data

    @param shotchart_df (DataFrame): DataFrame containing
    shotchart data

    Returns:

        shotchart_df (DataFrame): DataFrame containing
        the engineered features
    """

    shotchart_df['ANGLE'] = abs(np.rad2deg(np.arctan2(shotchart_df['LOC_X'],
                                                      shotchart_df['LOC_Y'])))

    shotchart_df['SIDE'] = [1 if x >= 0 else 0 for x in shotchart_df['LOC_X']]

    shotchart_df['DUNK'] = [1 if 'Dunk' in x else 0 for x in shotchart_df['ACTION_TYPE']]
    shotchart_df['HOOK'] = [1 if 'Hook' in x else 0 for x in shotchart_df['ACTION_TYPE']]
    shotchart_df['LAYUP'] = [1 if 'Layup' in x else 0 for x in shotchart_df['ACTION_TYPE']]
    shotchart_df['JUMP'] = [1 if 'Jump' in x else 0 for x in shotchart_df['ACTION_TYPE']]

    shotchart_df = shotchart_df[['SHOT_DISTANCE', 'ANGLE', 'SIDE', 'DUNK', 'HOOK', 'LAYUP', 'JUMP','SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG']]

    return shotchart_df

In [7]:
shotchart = feature_engineering(shot_chart)

In [8]:
shotchart

,SHOT_DISTANCE,ANGLE,SIDE,DUNK,HOOK,LAYUP,JUMP,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG
0,12,67.036227,0,0,0,0,0,1,0
1,13,65.376435,1,0,0,0,1,1,1
2,14,20.323137,1,0,0,0,1,1,0
3,0,0.000000,1,0,0,1,0,1,1
4,23,78.074008,0,0,0,0,1,1,1
...,...,...,...,...,...,...,...,...,...
217215,1,74.744881,1,1,0,0,0,1,1
217216,23,69.443955,1,0,0,0,1,1,1
217217,23,57.659407,1,0,0,0,1,1,0
217218,8,53.781163,0,0,0,0,1,1,0


In [9]:
shotchart['SHOT_MADE_FLAG'].value_counts()

SHOT_MADE_FLAG
0    113960
1    103260
Name: count, dtype: int64

Checking the split of which shots are being made. Seems to display almost even split

Next splitting the data into test and training sets

In [10]:
output = list(shotchart['SHOT_MADE_FLAG'])
shotchart = shotchart.drop(['SHOT_MADE_FLAG'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(shotchart, output, test_size=0.1, random_state=489)

### Model Building

#### Gradient Boosting

-Supervised Learning relies of labeled data, 
-Classification Problems, shot made or not
-AUC, area under ROC curve to evaluate, higher AUC is better model
-Features are numeric or categorical 
-Numeric Features should be scaled
-Categorial Variables are Encoded
-XGBoost
    - optimized gradient boosting machine
    - Speed
    - What is a decision tree?
        -single question at each tree and two choices
        -base learner, individual learner in an ensemble algorithm
        -constructed iteratively
        -individual trees tend to overfit
    -Classification and Regression Tree
        -each leaf always contains real value scores
    -Boosting
        -Ensemble algo to convert many weak learnerd to strong learners
        -Weak learner, any slightly better than 50%
        -boosting combines each weak learners prediction times their weights
        -


In [11]:
# Fixed Effects Gradient Boosted Trees
import xgboost as xgb


In [12]:
#Instantiate the XGBClassifier
xg_cl = xgb.XGBClassifier(objective='binary:logistic',n_estimators=10,seed=123)

#fit the classifier to the training set
xg_cl.fit(X_train,y_train)

#Predict the labels
preds=xg_cl.predict(X_test)

#compute the accuracy
accuracy = float(np.sum(preds==y_test))/np.shape(y_test)[0]
print("accuracy: %f" % (accuracy))

accuracy: 0.627520


In [13]:
#Create the DMatrix from X and y: churn_dmatrix
churn_dmatrix = xgb.DMatrix(data = X_train,label=y_train)

#Create the paramter dictionary: params
params = ("objective":"reg:logistic","max_depth":3}

SyntaxError: closing parenthesis '}' does not match opening parenthesis '(' (2607431374.py, line 5)

In [23]:
from pymer4.models import Lmer

model = Lmer("ANGLE  + (1|PLAYER_ID)",
             data=shotchart, family = 'binomial')

print(model.fit())

PackageNotInstalledError: The R package "lme4" is not installed.

In [1]:
team_stats = team.Team(headers=HEADERS,
                           endpoint='teamgamelogs',
                           team_id = '1610612755',
                           season = '2022-23',
                           
                           measure_type = 'Base',
                           
                           league_id='00')

team_stats = pd.DataFrame(team_stats.data['TeamGameLogs'])

NameError: name 'team' is not defined

In [ ]:
games = team_stats['GAME_ID']

In [76]:

all_team_stats = pd.DataFrame({})    
for game in games:
    game_id = game
    team_stats = boxscore.BoxScore(headers=HEADERS,
                           endpoint='boxscoreadvancedv2',
                           game_id = game_id
                           )
    team_stats = pd.DataFrame(team_stats.data['PlayerStats'])
            
            
    all_team_stats = pd.concat([all_team_stats, team_stats], axis=0).reset_index(drop=True)

    
            


KeyboardInterrupt: 

In [ ]:
team_stats = team.Team(headers=HEADERS,
                           endpoint='teamgamelogs',
                           team_id = '1610612755',
                           season = '2022-23',
                           
                           measure_type = 'Base',
                           
                           league_id='00')

        team_stats = pd.DataFrame(team_stats.data['TeamGameLogs'])

        team_games = team_stats['GAME_ID']
        all_team_stats = pd.DataFrame({})

        for game in team_games:
            game_id = game
            team_stats = boxscore.BoxScore(headers=HEADERS,
                           endpoint='boxscoreadvancedv2',
                           game_id = game_id
                           )
            team_stats = pd.DataFrame(team_stats.data['PlayerStats'])

In [11]:
team_stats = team.Team(headers=HEADERS,
                           endpoint='teamgamelogs',
                           team_id = '1610612755',
                           season = '2022-23',
                           
                           measure_type = 'Base',
                           
                           league_id='00')

team_stats = pd.DataFrame(team_stats.data['TeamGameLogs'])

team_games = team_stats['GAME_ID']
all_team_stats = pd.DataFrame({})